[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GabbyTab/boofun/blob/main/notebooks/lecture8_learning_juntas.ipynb)

# Lecture 8: Learning Juntas

**Topics**: Junta testing, Junta learning, Relevant variables

**References**: [Mossel-O'Donnell-Servedio'04], [Valiant'12]  
**Based on lecture notes by: Timothe Kasriel**  
**Notebook by: Gabriel Taboada**

---

## Key Concepts

| Term | Definition |
|------|------------|
| **$k$-Junta** | Function depending on **at most** $k$ variables |
| **Relevant variable** | Variable $i$ where $\text{Inf}_i[f] > 0$ |
| **Junta Testing** | Determine if $f$ is close to a $k$-junta |
| **Junta Learning** | Learn *which* $k$ variables matter |

**Key insight**: A function is a $k$-junta iff it has at most $k$ relevant variables.

In [1]:
# Install/upgrade boofun (required for Colab)
!pip install --upgrade boofun -q

import boofun as bf
print(f"BooFun version: {bf.__version__}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


/Users/gabrieltaboada/dev/Boofun/boofun/src/boofun/core/errormodels.py:21: UserWarning: uncertainties library not available - some error models disabled
  warnings.warn("uncertainties library not available - some error models disabled")


/Users/gabrieltaboada/dev/Boofun/boofun/src/boofun/quantum/__init__.py:22: UserWarning: Qiskit not available - quantum features limited
  warnings.warn("Qiskit not available - quantum features limited")


BooFun version: 1.1.1


In [2]:
import numpy as np
import boofun as bf

# Example functions
dictator = bf.dictator(8, 0)  # f(x) = x_0 (1 relevant variable)
maj7 = bf.majority(7)         # Majority on 7 variables (ALL 7 relevant)
and4 = bf.AND(4)              # AND of 4 variables (all 4 relevant)

print("Testing k-junta properties:")
print("="*55)

# Dictator: only x_0 matters → 1-junta
print(f"\nDictator-8: f(x) = x_0")
print(f"  Influences: {[f'{inf:.2f}' for inf in dictator.influences()]}")
print(f"  is_junta(1): {dictator.is_junta(1)}")

# Majority-7: ALL variables matter → exactly 7-junta
print(f"\nMajority-7: f(x) = sign(x_0 + ... + x_6)")
print(f"  Influences: {[f'{inf:.2f}' for inf in maj7.influences()]}")
print(f"  is_junta(3): {maj7.is_junta(3)}  ← NOT a 3-junta (needs all 7!)")
print(f"  is_junta(6): {maj7.is_junta(6)}  ← NOT a 6-junta")
print(f"  is_junta(7): {maj7.is_junta(7)}  ← IS a 7-junta")

# AND-4: all variables matter
print(f"\nAND-4: f(x) = x_0 ∧ x_1 ∧ x_2 ∧ x_3")
print(f"  Influences: {[f'{inf:.2f}' for inf in and4.influences()]}")
print(f"  is_junta(3): {and4.is_junta(3)}")
print(f"  is_junta(4): {and4.is_junta(4)}")

Testing k-junta properties:

Dictator-8: f(x) = x_0
  Influences: ['1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
  is_junta(1): True

Majority-7: f(x) = sign(x_0 + ... + x_6)
  Influences: ['0.31', '0.31', '0.31', '0.31', '0.31', '0.31', '0.31']
  is_junta(3): False  ← NOT a 3-junta (needs all 7!)
  is_junta(6): False  ← NOT a 6-junta
  is_junta(7): True  ← IS a 7-junta

AND-4: f(x) = x_0 ∧ x_1 ∧ x_2 ∧ x_3
  Influences: ['0.12', '0.12', '0.12', '0.12']
  is_junta(3): False
  is_junta(4): True


---
## Summary

- **$k$-Junta**: Depends on at most $k$ variables (has ≤ $k$ relevant variables)
- **Test**: Check if total influence on non-junta variables is small
- **Learn**: Use Goldreich-Levin to find relevant variables via heavy Fourier coefficients

**Runtime** ([MOS04]): $\approx n^{k\omega/(\omega+1)}$ where $\omega \approx 2.37$ is matrix multiplication exponent

```python
f.is_junta(k)        # Check if f is a k-junta
f.influences()       # Per-variable influences
f.total_influence()  # Sum of all influences
```